In [1]:
import numpy as np
import time
kkk = 0

In [2]:
def generate_matrix_and_vector(n, dtype=np.float64):
    matrix = np.zeros((n, n), dtype=dtype)

    matrix[0, :] = 1

    for i in range(1, n):
        for j in range(n):
            matrix[i, j] = 1 / (i + j + 1)

    vector = np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=dtype)

    return matrix, vector

In [3]:
def generate_matrix_and_vector_2(n, dtype=np.float64):
    matrix = np.zeros((n, n), dtype=dtype)

    for i in range(n):
        for j in range(i, n):
            matrix[i, j] = (2 * (i + 1)) / (j + 1)
            matrix[j, i] = matrix[i, j]

    vector = np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=dtype)

    return matrix, vector

In [4]:
def generate_matrix_and_vector_3(n, dtype=np.float64):
    matrix = np.zeros((n,n), dtype=dtype)
    m = 3
    k = 7
    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i, j] = -m * (i+1) - k
            elif j == i+1:
                matrix[i, j] = i + 1
            elif i > 0 and j == i-1:
                matrix[i, j] = m/(i+1)

    vector = np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=dtype)

    return matrix, vector

In [5]:
def generate_thomas_matrix(n, dtype=np.float64):
    matrix = np.zeros((3, n), dtype=dtype)
    m = 3
    k = 7
    for i in range(n):
        matrix[0, i] = m/(i+1) 
        matrix[1, i] = -m * (i+1) - k
        matrix[2, i] = i + 1
    
    matrix[0, 0] = 0
    matrix[-1, -1] = 0

    return matrix      

In [6]:
def solve(n, A, B, f_type=np.float64):
    for i in range(n):
        for j in range(i+1, n):
            ratio = A[j, i] / A[i, i]
            A[j] -= ratio * A[i]
            B[j] -= ratio * B[i]
    x_new = np.zeros(n, dtype=f_type)
    for i in range(n - 1, -1, -1):
        x_new[i] = (B[i] - np.dot(A[i, i + 1:], x_new[i + 1:])) / A[i, i]

    return x_new

In [7]:
def solve_thomas(n, f_type):
    A, x = generate_matrix_and_vector_3(n, f_type)
    B = (A @ x).astype(f_type)
    d = B.copy()
    time_g_start = time.time()
    x_new = solve(n, A, B, f_type)
    time_g_end = time.time()
    
    abc = generate_thomas_matrix(n, f_type)
    time_t_start = time.time()
    C = np.zeros(n, dtype=f_type)
    C[0] = abc[1, 0]
    X = np.zeros(n, dtype=f_type)
    X[0] = d[0]
    
    for i in range(1, n):
        factor = abc[0, i] / C[i - 1]
        C[i] = abc[1, i] - factor * abc[2, i - 1]
        X[i] = d[i] - factor * X[i - 1]
        
    X[-1] = X[-1] / C[-1]
    for i in range(n - 2, -1, -1):
        X[i] = (X[i] - abc[2, i] * X[i + 1]) / C[i]
    time_t_end = time.time()
    return x, x_new, X, time_g_end - time_g_start, time_t_end - time_t_start

In [8]:
def difference(v1, v2):
    differences = np.abs(v1 - v2)

    max_difference = np.max(differences)
    return max_difference

In [9]:
def generate_table():
    n_values = list(range(3, 31))

    float_types = [np.float16, np.float32, np.float64]

    results = {f_type: [] for f_type in float_types}

    for f_type in float_types:
        for n in n_values:
            A, x = generate_matrix_and_vector(n, f_type)
            B = (A @ x).astype(f_type)
            x_new = solve(n, A, B, f_type)
            diff = difference(x, x_new)
            results[f_type].append(f"{diff:.8f}")

    # Generate LaTeX table
    latex_code = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    latex_code += "\\begin{tabular}{l|l|l|l}\n"
    latex_code += "n & float16 & float32 & float64 \\\\\n\\hline\n"

    for i in range(len(n_values)):
        n = n_values[i]
        float16_val = results[np.float16][i]
        float32_val = results[np.float32][i]
        float64_val = results[np.float64][i]
        row = [n, float16_val, float32_val, float64_val]
        latex_code += " & ".join(map(str, row)) + " \\\\\n"
        if i < len(n_values) - 1:
            latex_code += "\\hline\n"

    latex_code += "\\end{tabular}\n\\end{minipage}"

    print(latex_code)

In [10]:
generate_table()

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{l|l|l|l}
n & float16 & float32 & float64 \\
\hline
3 & 0.00000000 & 0.00000209 & 0.00000000 \\
\hline
4 & 0.00000000 & 0.00000012 & 0.00000000 \\
\hline
5 & 2.45703125 & 0.00021541 & 0.00000000 \\
\hline
6 & 2.85156250 & 0.02232707 & 0.00000000 \\
\hline
7 & 12.43750000 & 0.22568977 & 0.00000001 \\
\hline
8 & 1.97363281 & 6.13138866 & 0.00000008 \\
\hline
9 & 3.89648438 & 13.61607647 & 0.00000034 \\
\hline
10 & 2.67187500 & 6.49310207 & 0.00010529 \\
\hline
11 & 12.04687500 & 8.97018051 & 0.00764796 \\
\hline
12 & 20.21875000 & 13.90666676 & 0.71986393 \\
\hline
13 & 8.32812500 & 76.66101074 & 12.53222833 \\
\hline
14 & 2.16015625 & 3.12784433 & 11.99258477 \\
\hline
15 & 7.09375000 & 7.34304523 & 9.06911186 \\
\hline
16 & 15.92968750 & 9.04860497 & 15.63721333 \\
\hline
17 & 11.82812500 & 13.35541821 & 14.41392810 \\
\hline
18 & 4.98437500 & 50.20633698 & 13.50041866 \\
\hline
19 & 8.63281250 & 27.96179008 & 52.15635040 

In [11]:
def generate_table2():
    n_values = list(range(3, 31)) + [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

    float_types = [np.float16, np.float32, np.float64]

    results = {f_type: [] for f_type in float_types}

    for f_type in float_types:
        for n in n_values:
            A, x = generate_matrix_and_vector_2(n, f_type)
            B = (A @ x).astype(f_type)
            x_new = solve(n, A, B, f_type)
            diff = difference(x, x_new)
            formatted_diff = f"{diff:.10f}"
            results[f_type].append(formatted_diff)

    # Generate LaTeX table
    latex_code = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    latex_code += "\\begin{tabular}{l|l|l|l}\n"
    latex_code += "n & float16 & float32 & float64 \\\\\n\\hline\n"

    for i in range(len(n_values)):
        n = n_values[i]
        float16_val = results[np.float16][i]
        float32_val = results[np.float32][i]
        float64_val = results[np.float64][i]
        row = [n, float16_val, float32_val, float64_val]
        latex_code += " & ".join(map(str, row)) + " \\\\\n"
        if i < len(n_values) - 1:
            latex_code += "\\hline\n"

    latex_code += "\\end{tabular}\n\\end{minipage}"

    print(latex_code)

In [12]:
generate_table2()

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{l|l|l|l}
n & float16 & float32 & float64 \\
\hline
3 & 0.0009765625 & 0.0000001192 & 0.0000000000 \\
\hline
4 & 0.0019531250 & 0.0000001192 & 0.0000000000 \\
\hline
5 & 0.0009765625 & 0.0000001192 & 0.0000000000 \\
\hline
6 & 0.0029296875 & 0.0000002980 & 0.0000000000 \\
\hline
7 & 0.0063476562 & 0.0000010729 & 0.0000000000 \\
\hline
8 & 0.0073242188 & 0.0000011921 & 0.0000000000 \\
\hline
9 & 0.0117187500 & 0.0000010729 & 0.0000000000 \\
\hline
10 & 0.0117187500 & 0.0000030994 & 0.0000000000 \\
\hline
11 & 0.0107421875 & 0.0000027418 & 0.0000000000 \\
\hline
12 & 0.0136718750 & 0.0000051260 & 0.0000000000 \\
\hline
13 & 0.0258789062 & 0.0000066757 & 0.0000000000 \\
\hline
14 & 0.0166015625 & 0.0000073314 & 0.0000000000 \\
\hline
15 & 0.0190429688 & 0.0000074506 & 0.0000000000 \\
\hline
16 & 0.0410156250 & 0.0000054836 & 0.0000000000 \\
\hline
17 & 0.0634765625 & 0.0000059009 & 0.0000000000 \\
\hline
18 & 0.0576171875 & 0

In [13]:
def norm(A):
    n = len(A)
    result = 0
    for i in range(n):
        temp = 0
        for j in range(n):
            temp += abs(A[i, j])
        result = max(result, temp)
    return result

In [14]:
def uwarunkowanie(n, typed=1):
    if typed == 1:
        A, _ = generate_matrix_and_vector(n)
    else:
        A, _ = generate_matrix_and_vector_2(n)
    A_inv = np.linalg.inv(A)
    return norm(A_inv) * norm(A)

In [15]:
def generate_table3():
    n_values = list(range(3, 31)) + [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

    results1 = []
    results2 = []

    for n in n_values:
        cond1 = round(uwarunkowanie(n, 1))
        cond2 = round(uwarunkowanie(n, 2))
        results1.append(cond1)
        results2.append(cond2)

    # Generate LaTeX table
    latex_code = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    latex_code += "\\begin{tabular}{c|c|c}\n"
    latex_code += "n & Zadanie 1 & Zadanie 2 \\\\\n\\hline\n"

    for i in range(len(n_values)):
        n = n_values[i]
        cond1 = results1[i]
        cond2 = results2[i]
        row = [n, cond1, cond2]
        latex_code += " & ".join(map(str, row)) + " \\\\\n"
        if i < len(n_values) - 1:
            latex_code += "\\hline\n"

    latex_code += "\\end{tabular}\n\\end{minipage}"

    print(latex_code)

In [16]:
generate_table3()

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{c|c|c}
n & Zadanie 1 & Zadanie 2 \\
\hline
3 & 864 & 9 \\
\hline
4 & 37920 & 16 \\
\hline
5 & 1442000 & 27 \\
\hline
6 & 56344680 & 40 \\
\hline
7 & 2232478706 & 55 \\
\hline
8 & 81552932522 & 72 \\
\hline
9 & 2843171843290 & 92 \\
\hline
10 & 108960945817880 & 115 \\
\hline
11 & 3951758149661762 & 140 \\
\hline
12 & 136453553538479040 & 167 \\
\hline
13 & 2725392318632902656 & 197 \\
\hline
14 & 2745139521386732544 & 229 \\
\hline
15 & 10433483999586590720 & 263 \\
\hline
16 & 2895360864787874304 & 301 \\
\hline
17 & 13547271121323634688 & 340 \\
\hline
18 & 2213939234398773182464 & 382 \\
\hline
19 & 35706711768479363072 & 426 \\
\hline
20 & 12204247200534620160 & 473 \\
\hline
21 & 18981325683706511360 & 522 \\
\hline
22 & 87949760078083948544 & 573 \\
\hline
23 & 68289260706615672832 & 627 \\
\hline
24 & 15152016253574156288 & 684 \\
\hline
25 & 70359837126158475264 & 742 \\
\hline
26 & 362112977615711371264 & 804 \\


In [17]:
def generate_thomas_tables():
    ns = range(50, 1050, 50)
    f_types = [np.float32, np.float64]

    times = []
    errors = []

    for n in ns:
        row_time = [n]
        row_error = [n]
        thomas_times = []
        gauss_times = []
        thomas_errors = []
        gauss_errors = []

        for f_type in f_types:
            x, x_new, X, time_gauss, time_thomas = solve_thomas(n, f_type)
            thomas_times.append(time_thomas)
            gauss_times.append(time_gauss)
            thomas_errors.append(difference(x, X))
            gauss_errors.append(difference(x, x_new))

        row_time.extend(thomas_times)
        row_time.extend(gauss_times)
        row_error.extend(thomas_errors)
        row_error.extend(gauss_errors)

        times.append(row_time)
        errors.append(row_error)
    
    # Generowanie tabeli czasów
    latex_times = (
        "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{c c c c c}\n\\hline\n"
        "\\multirow{2}{*}{n} & \\multicolumn{2}{c}{Thomas} & \\multicolumn{2}{c}{Gauss} \\\\\n"
        "\\cline{2-5}\n"
        " & float32 & float64 & float32 & float64 \\\\\n\\hline\n"
    )
    
    for row in times:
        latex_times += " & ".join(map(str, row)) + " \\\\\n"
    
    latex_times += "\\hline\n\\end{tabular}\n\\caption{Czasy wykonania}\n\\end{table}\n"
    
    # Generowanie tabeli błędów
    latex_errors = (
        "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{c c c c c}\n\\hline\n"
        "\\multirow{2}{*}{n} & \\multicolumn{2}{c}{Thomas} & \\multicolumn{2}{c}{Gauss} \\\\\n"
        "\\cline{2-5}\n"
        " & float32 & float64 & float32 & float64 \\\\\n\\hline\n"
    )
    
    for row in errors:
        latex_errors += " & ".join(map(str, row)) + " \\\\\n"
    
    latex_errors += "\\hline\n\\end{tabular}\n\\caption{Błędy obliczeniowe}\n\\end{table}\n"
    
    return latex_times, latex_errors

In [18]:
z = generate_thomas_tables()
print(z[0])
print(z[1])

\begin{table}[h!]
\centering
\begin{tabular}{c c c c c}
\hline
\multirow{2}{*}{n} & \multicolumn{2}{c}{Thomas} & \multicolumn{2}{c}{Gauss} \\
\cline{2-5}
 & float32 & float64 & float32 & float64 \\
\hline
50 & 0.0 & 0.0 & 0.004998445510864258 & 0.003988981246948242 \\
100 & 0.0 & 0.0 & 0.015002727508544922 & 0.015999555587768555 \\
150 & 0.0010006427764892578 & 0.0009984970092773438 & 0.0365140438079834 & 0.03551173210144043 \\
200 & 0.0 & 0.0 & 0.06000232696533203 & 0.06302690505981445 \\
250 & 0.0 & 0.0005354881286621094 & 0.09552931785583496 & 0.09851479530334473 \\
300 & 0.0009996891021728516 & 0.0009839534759521484 & 0.14500927925109863 & 0.14122796058654785 \\
350 & 0.0 & 0.0009996891021728516 & 0.19175100326538086 & 0.2615995407104492 \\
400 & 0.0 & 0.0009984970092773438 & 0.2500593662261963 & 0.2581522464752197 \\
450 & 0.0009996891021728516 & 0.0009984970092773438 & 0.3134782314300537 & 0.32045412063598633 \\
500 & 0.0009996891021728516 & 0.00099945068359375 & 0.40335988998413